In [2]:
import findspark
findspark.init('/home/anycaroliny/spark-3.3.2-bin-hadoop3')
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('miss').getOrCreate()

23/03/14 11:42:09 WARN Utils: Your hostname, anycaroliny-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/03/14 11:42:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/14 11:42:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/14 11:42:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/03/14 11:42:11 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/03/14 11:42:11 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [8]:
df = spark.read.csv('ContainsNull.csv', inferSchema= True,header=True)

In [9]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [14]:
df.na.drop(thresh=2).show() # The row needs to have at least two non null values

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [22]:
# df.na.drop(how='any').show()
# df.na.drop(how='all').show()
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [23]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [30]:
# df.na.fill('FILL VALUE').show()
# df.na.fill(300).show()
# df.na.fill('No Name', subset=['Name']).show()
df.na.fill(300, subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|300.0|
|emp2| null|300.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [41]:
from pyspark.sql.functions import mean

In [43]:
mean_val = df.select(mean(df['Sales'])).collect()

In [59]:
# mean_val
# mean_val[0]
mean_sales = mean_val[0][0]

In [62]:
# df.na.fill(mean_sales, subset=['Sales']).show()
df.na.fill(mean_sales, ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [64]:
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0], ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

